In [1]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

In [2]:
app_train=pd.read_csv('./home_credit_data/app_train.csv')

In [3]:
app_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 123 entries, Unnamed: 0 to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float64(65), int64(42), object(16)
memory usage: 9.4+ MB


In [50]:
pd.set_option('display.max_rows', 200)
app_train.head().T

,0,1,2,3,4
Unnamed: 0,148432,125153,60085,51714,91980
SK_ID_CURR,272103,245137,169665,159887,206817
TARGET,1,1,1,1,1
NAME_CONTRACT_TYPE,Cash loans,Cash loans,Cash loans,Cash loans,Cash loans
CODE_GENDER,F,F,F,F,M
FLAG_OWN_CAR,N,N,N,N,N
FLAG_OWN_REALTY,N,Y,N,N,Y
CNT_CHILDREN,0,1,2,0,0
AMT_INCOME_TOTAL,180000,135000,112500,157500,202500
AMT_CREDIT,450000,521280,291384,1.08221e+06,545040


In [4]:
app_train_corr=app_train.corr()

In [5]:
abs(app_train_corr.TARGET).sort_values(ascending=False)[:50]

TARGET                         1.000000
EXT_SOURCE_3                   0.312811
EXT_SOURCE_1                   0.311958
EXT_SOURCE_2                   0.262925
DAYS_BIRTH                     0.142876
REGION_RATING_CLIENT_W_CITY    0.117141
REGION_RATING_CLIENT           0.112320
DAYS_LAST_PHONE_CHANGE         0.092998
DAYS_REGISTRATION              0.088131
REG_CITY_NOT_LIVE_CITY         0.087663
FLAG_DOCUMENT_3                0.086024
FLAG_EMP_PHONE                 0.085963
AMT_GOODS_PRICE                0.084407
DAYS_EMPLOYED                  0.083923
DAYS_ID_PUBLISH                0.081256
FLOORSMAX_AVG                  0.076801
FLOORSMAX_MEDI                 0.074953
FLOORSMAX_MODE                 0.074615
REG_CITY_NOT_WORK_CITY         0.074523
REGION_POPULATION_RELATIVE     0.069843
FLOORSMIN_AVG                  0.067685
AMT_CREDIT                     0.067649
FLOORSMIN_MEDI                 0.067091
FLOORSMIN_MODE                 0.060869
BASEMENTAREA_AVG               0.056984


In [5]:
def select_columns(data_frame, column_names):
    new_frame = data_frame.loc[:, column_names]
    return new_frame

In [6]:
selected_columns = ['SK_ID_CURR','TARGET','EXT_SOURCE_3','EXT_SOURCE_1','EXT_SOURCE_2','DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','DAYS_LAST_PHONE_CHANGE','DAYS_REGISTRATION','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','FLAG_EMP_PHONE','AMT_GOODS_PRICE','DAYS_EMPLOYED','DAYS_ID_PUBLISH','FLOORSMAX_AVG','FLOORSMAX_MEDI','FLOORSMAX_MODE','REG_CITY_NOT_WORK_CITY','REGION_POPULATION_RELATIVE','FLOORSMIN_AVG','AMT_CREDIT','FLOORSMIN_MEDI','FLOORSMIN_MODE','BASEMENTAREA_AVG','BASEMENTAREA_MEDI','LIVINGAREA_AVG','FLAG_DOCUMENT_6','LIVINGAREA_MEDI','ELEVATORS_AVG','OWN_CAR_AGE','TOTALAREA_MODE','ELEVATORS_MEDI','ELEVATORS_MODE','DEF_30_CNT_SOCIAL_CIRCLE','LIVINGAREA_MODE','FLAG_WORK_PHONE','APARTMENTS_AVG','AMT_INCOME_TOTAL','APARTMENTS_MEDI','BASEMENTAREA_MODE','HOUR_APPR_PROCESS_START','ENTRANCES_MODE','ENTRANCES_MEDI','ENTRANCES_AVG','APARTMENTS_MODE','NONLIVINGAREA_MEDI','NONLIVINGAREA_AVG','DEF_60_CNT_SOCIAL_CIRCLE','NONLIVINGAREA_MODE']
train_model = select_columns(app_train, selected_columns)

In [7]:
train_model.dtypes[train_model.dtypes==object].index

Index([], dtype='object')

In [8]:
grouped=train_model.groupby(train_model.TARGET)

target_zero=grouped.get_group(0)
target_zero=pd.DataFrame(target_zero)
target_one=grouped.get_group(1)
target_one=pd.DataFrame(target_one)

In [9]:
from my_functions import missing_value

In [10]:
missing_value(target_zero)

,Missing Values,% of Total Values
FLOORSMIN_MODE,3357,67.1
FLOORSMIN_MEDI,3357,67.1
FLOORSMIN_AVG,3357,67.1
OWN_CAR_AGE,3333,66.7
BASEMENTAREA_MEDI,2903,58.1
BASEMENTAREA_MODE,2903,58.1
BASEMENTAREA_AVG,2903,58.1
EXT_SOURCE_1,2820,56.4
NONLIVINGAREA_MODE,2755,55.1
NONLIVINGAREA_MEDI,2755,55.1


In [11]:
target_zero['EXT_SOURCE_3'].fillna((target_zero['EXT_SOURCE_3'].median()), inplace=True)
target_zero['EXT_SOURCE_1'].fillna((target_zero['EXT_SOURCE_1'].median()), inplace=True)
target_zero['EXT_SOURCE_2'].fillna((target_zero['EXT_SOURCE_2'].median()), inplace=True)
target_zero['AMT_GOODS_PRICE'].fillna((target_zero['AMT_GOODS_PRICE'].median()), inplace=True)
target_zero['FLOORSMAX_AVG'].fillna((target_zero['FLOORSMAX_AVG'].median()), inplace=True)
target_zero['FLOORSMAX_MEDI'].fillna((target_zero['FLOORSMAX_MEDI'].median()), inplace=True)
target_zero['FLOORSMAX_MEDI'].fillna((target_zero['FLOORSMAX_MEDI'].median()), inplace=True)
target_zero['FLOORSMIN_AVG'].fillna((target_zero['FLOORSMIN_AVG'].median()), inplace=True)
target_zero['FLOORSMIN_MEDI'].fillna((target_zero['FLOORSMIN_MEDI'].median()), inplace=True)
target_zero['FLOORSMAX_MODE'].fillna((target_zero['FLOORSMAX_MODE'].median()), inplace=True)
target_zero['FLOORSMIN_MODE'].fillna((target_zero['FLOORSMIN_MODE'].median()), inplace=True)
target_zero['BASEMENTAREA_AVG'].fillna((target_zero['BASEMENTAREA_AVG'].median()), inplace=True)
target_zero['BASEMENTAREA_MEDI'].fillna((target_zero['BASEMENTAREA_MEDI'].median()), inplace=True)
target_zero['LIVINGAREA_AVG'].fillna((target_zero['LIVINGAREA_AVG'].median()), inplace=True)
target_zero['LIVINGAREA_MEDI'].fillna((target_zero['LIVINGAREA_MEDI'].median()), inplace=True)
target_zero['ELEVATORS_AVG'].fillna((target_zero['ELEVATORS_AVG'].median()), inplace=True)
target_zero['OWN_CAR_AGE'].fillna((target_zero['OWN_CAR_AGE'].median()), inplace=True)
target_zero['TOTALAREA_MODE'].fillna((target_zero['TOTALAREA_MODE'].median()), inplace=True)
target_zero['ELEVATORS_MEDI'].fillna((target_zero['ELEVATORS_MEDI'].median()), inplace=True)
target_zero['ELEVATORS_MODE'].fillna((target_zero['ELEVATORS_MODE'].median()), inplace=True)
target_zero['DEF_30_CNT_SOCIAL_CIRCLE'].fillna((target_zero['DEF_30_CNT_SOCIAL_CIRCLE'].median()), inplace=True)
target_zero['LIVINGAREA_MODE'].fillna((target_zero['LIVINGAREA_MODE'].median()), inplace=True)
target_zero['APARTMENTS_AVG'].fillna((target_zero['APARTMENTS_AVG'].median()), inplace=True)
target_zero['APARTMENTS_MEDI'].fillna((target_zero['APARTMENTS_MEDI'].median()), inplace=True)
target_zero['BASEMENTAREA_MODE'].fillna((target_zero['BASEMENTAREA_MODE'].median()), inplace=True)
target_zero['ENTRANCES_MODE'].fillna((target_zero['ENTRANCES_MODE'].median()), inplace=True)
target_zero['ENTRANCES_MEDI'].fillna((target_zero['ENTRANCES_MEDI'].median()), inplace=True)
target_zero['ENTRANCES_AVG'].fillna((target_zero['ENTRANCES_AVG'].median()), inplace=True)
target_zero['APARTMENTS_MODE'].fillna((target_zero['APARTMENTS_MODE'].median()), inplace=True)
target_zero['NONLIVINGAREA_MEDI'].fillna((target_zero['NONLIVINGAREA_MEDI'].median()), inplace=True)
target_zero['NONLIVINGAREA_AVG'].fillna((target_zero['NONLIVINGAREA_AVG'].median()), inplace=True)
target_zero['DEF_60_CNT_SOCIAL_CIRCLE'].fillna((target_zero['DEF_60_CNT_SOCIAL_CIRCLE'].median()), inplace=True)
target_zero['NONLIVINGAREA_MODE'].fillna((target_zero['NONLIVINGAREA_MODE'].median()), inplace=True)

In [12]:
missing_value(target_one)

,Missing Values,% of Total Values
FLOORSMIN_MODE,3655,73.1
FLOORSMIN_AVG,3655,73.1
FLOORSMIN_MEDI,3655,73.1
OWN_CAR_AGE,3476,69.5
BASEMENTAREA_MEDI,3263,65.3
BASEMENTAREA_AVG,3263,65.3
BASEMENTAREA_MODE,3263,65.3
NONLIVINGAREA_MODE,3104,62.1
NONLIVINGAREA_MEDI,3104,62.1
NONLIVINGAREA_AVG,3104,62.1


In [13]:
target_one['EXT_SOURCE_3'].fillna((target_one['EXT_SOURCE_3'].median()), inplace=True)
target_one['EXT_SOURCE_1'].fillna((target_one['EXT_SOURCE_1'].median()), inplace=True)
target_one['EXT_SOURCE_2'].fillna((target_one['EXT_SOURCE_2'].median()), inplace=True)
target_one['AMT_GOODS_PRICE'].fillna((target_one['AMT_GOODS_PRICE'].median()), inplace=True)
target_one['FLOORSMAX_AVG'].fillna((target_one['FLOORSMAX_AVG'].median()), inplace=True)
target_one['FLOORSMAX_MEDI'].fillna((target_one['FLOORSMAX_MEDI'].median()), inplace=True)
target_one['FLOORSMAX_MODE'].fillna((target_one['FLOORSMAX_MODE'].median()), inplace=True)
target_one['FLOORSMIN_AVG'].fillna((target_one['FLOORSMIN_AVG'].median()), inplace=True)
target_one['FLOORSMIN_MEDI'].fillna((target_one['FLOORSMIN_MEDI'].median()), inplace=True)
target_one['FLOORSMIN_MEDI'].fillna((target_one['FLOORSMIN_MEDI'].median()), inplace=True)
target_one['FLOORSMIN_MODE'].fillna((target_one['FLOORSMIN_MODE'].median()), inplace=True)
target_one['BASEMENTAREA_AVG'].fillna((target_one['BASEMENTAREA_AVG'].median()), inplace=True)
target_one['BASEMENTAREA_MEDI'].fillna((target_one['BASEMENTAREA_MEDI'].median()), inplace=True)
target_one['LIVINGAREA_AVG'].fillna((target_one['LIVINGAREA_AVG'].median()), inplace=True)
target_one['LIVINGAREA_MEDI'].fillna((target_one['LIVINGAREA_MEDI'].median()), inplace=True)
target_one['ELEVATORS_AVG'].fillna((target_one['ELEVATORS_AVG'].median()), inplace=True)
target_one['OWN_CAR_AGE'].fillna((target_one['OWN_CAR_AGE'].median()), inplace=True)
target_one['TOTALAREA_MODE'].fillna((target_one['TOTALAREA_MODE'].median()), inplace=True)
target_one['ELEVATORS_MEDI'].fillna((target_one['ELEVATORS_MEDI'].median()), inplace=True)
target_one['ELEVATORS_MODE'].fillna((target_one['ELEVATORS_MODE'].median()), inplace=True)
target_one['DEF_30_CNT_SOCIAL_CIRCLE'].fillna((target_one['DEF_30_CNT_SOCIAL_CIRCLE'].median()), inplace=True)
target_one['LIVINGAREA_MODE'].fillna((target_one['LIVINGAREA_MODE'].median()), inplace=True)
target_one['APARTMENTS_AVG'].fillna((target_one['APARTMENTS_AVG'].median()), inplace=True)
target_one['APARTMENTS_MEDI'].fillna((target_one['APARTMENTS_MEDI'].median()), inplace=True)
target_one['BASEMENTAREA_MODE'].fillna((target_one['BASEMENTAREA_MODE'].median()), inplace=True)
target_one['ENTRANCES_MODE'].fillna((target_one['ENTRANCES_MODE'].median()), inplace=True)
target_one['ENTRANCES_MEDI'].fillna((target_one['ENTRANCES_MEDI'].median()), inplace=True)
target_one['ENTRANCES_AVG'].fillna((target_one['ENTRANCES_AVG'].median()), inplace=True)
target_one['APARTMENTS_MODE'].fillna((target_one['APARTMENTS_MODE'].median()), inplace=True)
target_one['NONLIVINGAREA_MEDI'].fillna((target_one['NONLIVINGAREA_MEDI'].median()), inplace=True)
target_one['NONLIVINGAREA_AVG'].fillna((target_one['NONLIVINGAREA_AVG'].median()), inplace=True)
target_one['DEF_60_CNT_SOCIAL_CIRCLE'].fillna((target_one['DEF_60_CNT_SOCIAL_CIRCLE'].median()), inplace=True)
target_one['NONLIVINGAREA_MODE'].fillna((target_one['NONLIVINGAREA_MODE'].median()), inplace=True)

In [14]:
train_model=pd.concat([target_one,target_zero])

## First Model Logistic Regression

In [15]:
Y=train_model['TARGET']
X=train_model[['EXT_SOURCE_3','EXT_SOURCE_1','EXT_SOURCE_2','DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','DAYS_LAST_PHONE_CHANGE','DAYS_REGISTRATION','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','FLAG_EMP_PHONE','AMT_GOODS_PRICE','DAYS_EMPLOYED','DAYS_ID_PUBLISH','FLOORSMAX_AVG','FLOORSMAX_MEDI','FLOORSMAX_MODE','REG_CITY_NOT_WORK_CITY','REGION_POPULATION_RELATIVE','FLOORSMIN_AVG','AMT_CREDIT','FLOORSMIN_MEDI','FLOORSMIN_MODE','BASEMENTAREA_AVG','BASEMENTAREA_MEDI','LIVINGAREA_AVG','FLAG_DOCUMENT_6','LIVINGAREA_MEDI','ELEVATORS_AVG','OWN_CAR_AGE','TOTALAREA_MODE','ELEVATORS_MEDI','ELEVATORS_MODE','DEF_30_CNT_SOCIAL_CIRCLE','LIVINGAREA_MODE','FLAG_WORK_PHONE','APARTMENTS_AVG','AMT_INCOME_TOTAL','APARTMENTS_MEDI','BASEMENTAREA_MODE','HOUR_APPR_PROCESS_START','ENTRANCES_MODE','ENTRANCES_MEDI','ENTRANCES_AVG','APARTMENTS_MODE','NONLIVINGAREA_MEDI','NONLIVINGAREA_AVG','DEF_60_CNT_SOCIAL_CIRCLE','NONLIVINGAREA_MODE']]

lrm=linear_model.LogisticRegression()
lrm.fit(X,Y)

LogisticRegression()

In [16]:
import statsmodels.api as sm

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 TARGET   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     120.1
Date:                Tue, 18 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:50:11   Log-Likelihood:                -4935.1
No. Observations:               10000   AIC:                             9970.
Df Residuals:                    9950   BIC:                         1.033e+04
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -2.3174      0.823     -2.815      0.005      -3.931      -0.704
EXT_SOURCE_3                   -0.6254      0.022    -27.989      0.000      -0.669      -0.582
EXT_SOURCE_1                   -1.3962      0.029    -48.616      0.000      -1.453      -1.340
EXT_SOURCE_2                   -0.3210      0.021    -15.117      0.000      -0.363      -0.279
DAYS_BIRTH                  -1.598e-05   1.34e-06    -11.932      0.000   -1.86e-05   -1.34e-05
REGION_RATING_CLIENT_W_CITY     0.0685      0.026      2.599      0.009       0.017       0.120
REGION_RATING_CLIENT           -0.0423      0.026     -1.625      0.104      -0.093       0.009
DAYS_LAST_PHONE_CHANGE      -3.197e-06    5.2e-06     -0.615      0.539   -1.34e-05       7e-06
DAYS_REGISTRATION            3.071e-06   1.25e-06      2.456      0.014     6.2e-07    5.52e-06
REG_CITY_NOT_LIVE_CITY          0.0420      0.015      2.812      0.005       0.013       0.071
FLAG_DOCUMENT_3                 0.0450      0.011      4.246      0.000       0.024       0.066
FLAG_EMP_PHONE                  3.5202      0.819      4.296      0.000       1.914       5.126
AMT_GOODS_PRICE             -3.051e-07   6.94e-08     -4.398      0.000   -4.41e-07   -1.69e-07
DAYS_EMPLOYED                9.359e-06   2.22e-06      4.212      0.000       5e-06    1.37e-05
DAYS_ID_PUBLISH              2.524e-07   2.78e-06      0.091      0.928    -5.2e-06     5.7e-06
FLOORSMAX_AVG                   0.4314      0.748      0.577      0.564      -1.035       1.898
FLOORSMAX_MEDI                  0.4403      0.794      0.555      0.579      -1.116       1.997
FLOORSMAX_MODE                 -0.7419      0.318     -2.334      0.020      -1.365      -0.119
REG_CITY_NOT_WORK_CITY         -0.0222      0.011     -2.095      0.036      -0.043      -0.001
REGION_POPULATION_RELATIVE      0.1234      0.371      0.332      0.740      -0.605       0.852
FLOORSMIN_AVG                  -0.2188      0.868     -0.252      0.801      -1.919       1.482
AMT_CREDIT                   2.799e-07   6.29e-08      4.448      0.000    1.57e-07    4.03e-07
FLOORSMIN_MEDI                 -0.8339      0.933     -0.894      0.371      -2.663       0.995
FLOORSMIN_MODE                  1.0605      0.414      2.561      0.010       0.249       1.872
BASEMENTAREA_AVG               12.7323      1.724      7.385      0.000       9.353      16.112
BASEMENTAREA_MEDI             -12.5679      1.696     -7.411      0.000     -15.892      -9.244
LIVINGAREA_AVG                 -3.1192      0.869     -3.589      0.000      -4.823      -1.416
FLAG_DOCUMENT_6                 0.0128      0.021      0.600      0.549      -0.029       0.055
LIVINGAREA_MEDI                 0.1150      0.890      0.129      0.897      -1.629       1.859
ELEVATORS_A

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.20, random_state=111)

In [18]:
first_model = LogisticRegression()
first_model.fit(X_train, y_train)

prediction_train = first_model.predict(X_train)
prediction_test = first_model.predict(X_test)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [19]:
from sklearn.metrics import confusion_matrix
hata_matrisi_eğitim = confusion_matrix(y_train, prediction_train)
hata_matrisi_test = confusion_matrix(y_test, prediction_test)

In [20]:
print("Hata Matrisi (Eğitim verileri)", "-"*30, hata_matrisi_eğitim, sep="\n")
print("Hata Matrisi (Test verileri)", "-"*30, hata_matrisi_test, sep="\n")

Hata Matrisi (Eğitim verileri)
------------------------------
[[2254 1686]
 [1821 2239]]
Hata Matrisi (Test verileri)
------------------------------
[[620 440]
 [429 511]]


In [21]:
TN = hata_matrisi_test[0][0]
TP = hata_matrisi_test[1][1]
FP = hata_matrisi_test[0][1]
FN = hata_matrisi_test[1][0]

print("Doğru negatif sayısı   :", TN)
print("Doğru pozitif sayısı   :", TP)
print("Yanlış pozitif sayısı  :", FP)
print("Yanlış negatif sayısı  :", FN)

Doğru negatif sayısı   : 620
Doğru pozitif sayısı   : 511
Yanlış pozitif sayısı  : 440
Yanlış negatif sayısı  : 429


In [22]:
from sklearn.metrics import recall_score

print("recall_score() değeri        : ",  recall_score(y_test,prediction_test))

recall_score() değeri        :  0.5436170212765957


In [23]:
parameters = {"C": [10 ** x for x in range (-5, 5, 1)],
                "penalty": ['l2']
                }

In [24]:
from sklearn.model_selection import GridSearchCV
grid_cv = GridSearchCV(estimator=first_model,param_grid = parameters,cv = 10)
grid_cv.fit(X, Y)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-le

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100,
                               1000, 10000],
                         'penalty': ['l2']})

In [25]:
print("The best parameter is : ", grid_cv.best_params_)
print("The best score is         : ", grid_cv.best_score_)

The best parameter is :  {'C': 0.001, 'penalty': 'l2'}
The best score is         :  0.5617


## Second Model

In [26]:
model_logged=train_model.copy()

In [27]:
model_logged['EXT_SOURCE_1']=np.log(model_logged['EXT_SOURCE_1']+1)
model_logged['EXT_SOURCE_2']=np.log(model_logged['EXT_SOURCE_2']+1)
model_logged['DAYS_BIRTH']=np.log(model_logged['DAYS_BIRTH'].abs()+1)
model_logged['DAYS_LAST_PHONE_CHANGE']=np.log(model_logged['DAYS_LAST_PHONE_CHANGE'].abs()+1)
model_logged['DAYS_REGISTRATION']=np.log(model_logged['DAYS_REGISTRATION'].abs()+1)
model_logged['AMT_GOODS_PRICE']=np.log(model_logged['AMT_GOODS_PRICE']+1)
model_logged['DAYS_EMPLOYED']=np.log(model_logged['DAYS_EMPLOYED'].abs()+1)
model_logged['DAYS_ID_PUBLISH']=np.log(model_logged['DAYS_ID_PUBLISH'].abs()+1)
model_logged['FLOORSMAX_AVG']=np.log(model_logged['FLOORSMAX_AVG']+1)
model_logged['FLOORSMAX_MEDI']=np.log(model_logged['FLOORSMAX_MEDI']+1)
model_logged['FLOORSMAX_MODE']=np.log(model_logged['FLOORSMAX_MODE']+1)
model_logged['FLOORSMIN_AVG']=np.log(model_logged['FLOORSMIN_AVG']+1)
model_logged['FLOORSMIN_MEDI']=np.log(model_logged['FLOORSMIN_MEDI']+1)
model_logged['FLOORSMIN_MODE']=np.log(model_logged['FLOORSMIN_MODE']+1)
model_logged['BASEMENTAREA_AVG']=np.log(model_logged['BASEMENTAREA_AVG']+1)
model_logged['BASEMENTAREA_MEDI']=np.log(model_logged['BASEMENTAREA_MEDI']+1)
model_logged['LIVINGAREA_AVG']=np.log(model_logged['LIVINGAREA_AVG']+1)
model_logged['LIVINGAREA_MEDI']=np.log(model_logged['LIVINGAREA_MEDI']+1)
model_logged['ELEVATORS_AVG']=np.log(model_logged['ELEVATORS_AVG']+1)
model_logged['TOTALAREA_MODE']=np.log(model_logged['TOTALAREA_MODE']+1)
model_logged['ELEVATORS_MEDI']=np.log(model_logged['ELEVATORS_MEDI']+1)
model_logged['ELEVATORS_MODE']=np.log(model_logged['ELEVATORS_MODE']+1)
model_logged['LIVINGAREA_MODE']=np.log(model_logged['LIVINGAREA_MODE']+1)
model_logged['APARTMENTS_AVG']=np.log(model_logged['APARTMENTS_AVG']+1)
model_logged['APARTMENTS_MEDI']=np.log(model_logged['APARTMENTS_MEDI']+1)
model_logged['APARTMENTS_MODE']=np.log(model_logged['APARTMENTS_MODE']+1)
model_logged['BASEMENTAREA_MODE']=np.log(model_logged['BASEMENTAREA_MODE']+1)
model_logged['ENTRANCES_AVG']=np.log(model_logged['ENTRANCES_AVG']+1)
model_logged['ENTRANCES_MEDI']=np.log(model_logged['ENTRANCES_MEDI']+1)
model_logged['ENTRANCES_MODE']=np.log(model_logged['ENTRANCES_MODE']+1)

In [28]:
group_one=(train_model['DAYS_BIRTH'].mean()+train_model['DAYS_BIRTH'].min())/2
group_two=(train_model['DAYS_BIRTH'].mean()+train_model['DAYS_BIRTH'].max())/2
mean=train_model['DAYS_BIRTH'].mean()


In [29]:
def categorized_sales(x,group_one,group_two,mean):
    if x<mean:
        if x<group_one:
            return 1
        else:
            return 2
    else:
        if x<group_two:
            return 3
        else:
            return 4

In [30]:
model_logged['DAYS_BIRTH_GROUP']=train_model['DAYS_BIRTH'].apply(lambda x: categorized_sales(x,group_one,group_two,mean))


In [31]:
Y=model_logged['TARGET']
X=model_logged[['EXT_SOURCE_3','EXT_SOURCE_1','EXT_SOURCE_2','DAYS_BIRTH', 'REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','DAYS_LAST_PHONE_CHANGE','DAYS_REGISTRATION','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','FLAG_EMP_PHONE','AMT_GOODS_PRICE','DAYS_EMPLOYED','DAYS_ID_PUBLISH','FLOORSMAX_AVG','FLOORSMAX_MEDI','FLOORSMAX_MODE','REG_CITY_NOT_WORK_CITY','REGION_POPULATION_RELATIVE','FLOORSMIN_AVG','AMT_CREDIT','FLOORSMIN_MEDI','FLOORSMIN_MODE','BASEMENTAREA_AVG','BASEMENTAREA_MEDI','LIVINGAREA_AVG','FLAG_DOCUMENT_6','LIVINGAREA_MEDI','ELEVATORS_AVG','OWN_CAR_AGE','TOTALAREA_MODE','ELEVATORS_MEDI','ELEVATORS_MODE','DEF_30_CNT_SOCIAL_CIRCLE','LIVINGAREA_MODE','FLAG_WORK_PHONE','APARTMENTS_AVG','AMT_INCOME_TOTAL','APARTMENTS_MEDI','BASEMENTAREA_MODE','HOUR_APPR_PROCESS_START','ENTRANCES_MODE','ENTRANCES_MEDI','ENTRANCES_AVG','APARTMENTS_MODE','NONLIVINGAREA_MEDI','NONLIVINGAREA_AVG','DEF_60_CNT_SOCIAL_CIRCLE','NONLIVINGAREA_MODE','DAYS_BIRTH_GROUP']]

In [32]:
lrm=linear_model.LogisticRegression()
lrm.fit(X,Y)

LogisticRegression()

In [33]:
import statsmodels.api as sm

X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 TARGET   R-squared:                       0.376
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     119.9
Date:                Tue, 18 Aug 2020   Prob (F-statistic):               0.00
Time:                        15:50:46   Log-Likelihood:                -4900.1
No. Observations:               10000   AIC:                             9902.
Df Residuals:                    9949   BIC:                         1.027e+04
Df Model:                          50                                         
Covariance Type:            nonrobust                                         
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const                          -1.9741      0.524     -3.769      0.000      -3.001      -0.947
EXT_SOURCE_3                   -0.6254      0.022    -28.146      0.000      -0.669      -0.582
EXT_SOURCE_1                   -2.0333      0.041    -49.356      0.000      -2.114      -1.953
EXT_SOURCE_2                   -0.4498      0.029    -15.366      0.000      -0.507      -0.392
DAYS_BIRTH                      0.3341      0.049      6.873      0.000       0.239       0.429
REGION_RATING_CLIENT_W_CITY     0.0703      0.026      2.676      0.007       0.019       0.122
REGION_RATING_CLIENT           -0.0432      0.026     -1.662      0.096      -0.094       0.008
DAYS_LAST_PHONE_CHANGE         -0.0011      0.002     -0.673      0.501      -0.004       0.002
DAYS_REGISTRATION              -0.0066      0.003     -1.955      0.051      -0.013     1.6e-05
REG_CITY_NOT_LIVE_CITY          0.0460      0.015      3.086      0.002       0.017       0.075
FLAG_DOCUMENT_3                 0.0496      0.010      4.743      0.000       0.029       0.070
FLAG_EMP_PHONE                 -0.0327      0.028     -1.166      0.244      -0.088       0.022
AMT_GOODS_PRICE                 0.0413      0.015      2.834      0.005       0.013       0.070
DAYS_EMPLOYED                  -0.0182      0.004     -4.136      0.000      -0.027      -0.010
DAYS_ID_PUBLISH                -0.0042      0.004     -0.999      0.318      -0.012       0.004
FLOORSMAX_AVG                   0.8463      0.994      0.851      0.395      -1.102       2.795
FLOORSMAX_MEDI                  0.2173      1.054      0.206      0.837      -1.850       2.284
FLOORSMAX_MODE                 -0.8586      0.405     -2.120      0.034      -1.653      -0.065
REG_CITY_NOT_WORK_CITY         -0.0229      0.011     -2.162      0.031      -0.044      -0.002
REGION_POPULATION_RELATIVE      0.1500      0.369      0.406      0.685      -0.574       0.874
FLOORSMIN_AVG                  -0.5332      1.127     -0.473      0.636      -2.743       1.677
AMT_CREDIT                  -6.233e-08   2.67e-08     -2.335      0.020   -1.15e-07      -1e-08
FLOORSMIN_MEDI                 -0.6990      1.211     -0.577      0.564      -3.074       1.676
FLOORSMIN_MODE                  1.2590      0.507      2.481      0.013       0.264       2.254
BASEMENTAREA_AVG               14.9767      1.915      7.820      0.000      11.223      18.731
BASEMENTAREA_MEDI             -14.6091      1.870     -7.814      0.000     -18.274     -10.944
LIVINGAREA_AVG                 -5.0904      1.127     -4.516      0.000      -7.300      -2.881
FLAG_DOCUMENT_6                 0.0214      0.021      1.008      0.313      -0.020       0.063
LIVINGAREA_MEDI                 1.3574      1.168      1.162      0.245      -0.932       3.647
ELEVATORS_A

## Feature Engineering

### From Bureau

In [34]:
bureau = pd.read_csv('./home_credit_data/bureau_test.csv')
bureau.head()

,Unnamed: 0,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,33,400486,5714504,Closed,currency 1,-2413,0,-1864.0,-2128.0,NaN,0,180000.0,0.0,0.0,0.0,Consumer credit,-708,NaN
1,34,400486,5714506,Active,currency 1,-941,0,-17.0,NaN,NaN,0,40500.0,0.0,0.0,0.0,Credit card,-15,NaN
2,35,400486,5714507,Closed,currency 1,-1110,0,-745.0,-727.0,NaN,0,114727.5,0.0,0.0,0.0,Consumer credit,-658,NaN
3,36,400486,5714508,Closed,currency 1,-2133,0,-1038.0,-1127.0,NaN,0,225000.0,0.0,0.0,0.0,Consumer credit,-708,NaN
4,37,400486,5714509,Active,currency 1,-391,0,705.0,NaN,NaN,0,142290.0,108315.0,0.0,0.0,Consumer credit,-30,NaN


In [35]:
PREVIOUS_LOAN_COUNTS = bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'PREVIOUS_LOAN_COUNTS'})
PREVIOUS_LOAN_COUNTS.head()

,SK_ID_CURR,PREVIOUS_LOAN_COUNTS
0,100004,2
1,100054,2
2,100055,7
3,100077,4
4,100181,10


In [36]:
model_df = model_logged.merge(PREVIOUS_LOAN_COUNTS, on = 'SK_ID_CURR', how = 'left')
model_df.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,...,ENTRANCES_MODE,ENTRANCES_MEDI,ENTRANCES_AVG,APARTMENTS_MODE,NONLIVINGAREA_MEDI,NONLIVINGAREA_AVG,DEF_60_CNT_SOCIAL_CIRCLE,NONLIVINGAREA_MODE,DAYS_BIRTH_GROUP,PREVIOUS_LOAN_COUNTS
0,272103,1,0.209155,0.383783,0.498982,9.325899,2,2,7.750184,9.323758,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,4,9.0
1,245137,1,0.420611,0.176568,0.184767,9.226902,2,2,5.659482,8.420242,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,1.0,0.0000,4,3.0
2,169665,1,0.192942,0.298317,0.396168,9.846970,2,2,5.993961,9.166911,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,2,1.0
3,159887,1,0.646330,0.298317,0.437300,9.545526,2,2,2.079442,9.132811,...,0.129184,0.129184,0.129184,0.139936,0.0013,0.00120,0.0,0.0013,3,15.0
4,206817,1,0.466864,0.298317,0.151982,9.815093,2,2,7.296413,7.817223,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,2,15.0


In [37]:
model_df['PREVIOUS_LOAN_COUNTS'] = model_df['PREVIOUS_LOAN_COUNTS'].fillna(0)
model_df.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,...,ENTRANCES_MODE,ENTRANCES_MEDI,ENTRANCES_AVG,APARTMENTS_MODE,NONLIVINGAREA_MEDI,NONLIVINGAREA_AVG,DEF_60_CNT_SOCIAL_CIRCLE,NONLIVINGAREA_MODE,DAYS_BIRTH_GROUP,PREVIOUS_LOAN_COUNTS
0,272103,1,0.209155,0.383783,0.498982,9.325899,2,2,7.750184,9.323758,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,4,9.0
1,245137,1,0.420611,0.176568,0.184767,9.226902,2,2,5.659482,8.420242,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,1.0,0.0000,4,3.0
2,169665,1,0.192942,0.298317,0.396168,9.846970,2,2,5.993961,9.166911,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,2,1.0
3,159887,1,0.646330,0.298317,0.437300,9.545526,2,2,2.079442,9.132811,...,0.129184,0.129184,0.129184,0.139936,0.0013,0.00120,0.0,0.0013,3,15.0
4,206817,1,0.466864,0.298317,0.151982,9.815093,2,2,7.296413,7.817223,...,0.129184,0.129184,0.129184,0.072879,0.0012,0.00175,0.0,0.0000,2,15.0


In [38]:
def agg_numeric(df, group_var, df_name):
    """Aggregates the numeric values in a dataframe. This can
    be used to create features for each instance of the grouping variable.
    
    Parameters
    --------
        df (dataframe): 
            the dataframe to calculate the statistics on
        group_var (string): 
            the variable by which to group df
        df_name (string): 
            the variable used to rename the columns
        
    Return
    --------
        agg (dataframe): 
            a dataframe with the statistics aggregated for 
            all numeric columns. Each instance of the grouping variable will have 
            the statistics (mean, min, max, sum; currently supported) calculated. 
            The columns are also renamed to keep track of features created.
    
    """
    # Remove id variables other than grouping variable
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Group by the specified variable and calculate the statistics
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'sum','min','max']).reset_index()

    # Need to create new column names
    columns = [group_var]

    # Iterate through the variables names
    for var in agg.columns.levels[0]:
        # Skip the grouping variable
        if var != group_var:
            # Iterate through the stat names
            for stat in agg.columns.levels[1][:-1]:
                # Make a new column name for the variable and stat
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [39]:
bureau_agg = agg_numeric(df=bureau.drop(columns = ['SK_ID_BUREAU','CREDIT_ACTIVE','CREDIT_CURRENCY']), group_var = 'SK_ID_CURR', df_name = 'bureau')
bureau_agg.head()

<ipython-input-38-d564776ea464>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_df[group_var] = group_ids


,SK_ID_CURR,bureau_Unnamed: 0_count,bureau_Unnamed: 0_mean,bureau_Unnamed: 0_sum,bureau_Unnamed: 0_min,bureau_Unnamed: 0_max,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_sum,bureau_DAYS_CREDIT_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_sum,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_max
0,100004,2,363104.50,726209,363104,363105,2,-867.000000,-1734,-1326,...,2,-532.00,-1064,-682,-382,0,NaN,0.0,NaN,NaN
1,100054,2,83465.50,166931,83465,83466,2,-1310.500000,-2621,-1483,...,2,-578.00,-1156,-1114,-42,0,NaN,0.0,NaN,NaN
2,100055,7,854069.00,5978483,287930,948428,7,-876.857143,-6138,-2202,...,7,-288.00,-2016,-789,-13,0,NaN,0.0,NaN,NaN
3,100077,4,633170.75,2532683,274118,1710326,4,-824.250000,-3297,-1186,...,4,-537.75,-2151,-773,-24,0,NaN,0.0,NaN,NaN
4,100181,10,916160.50,9161605,916156,916165,10,-1041.300000,-10413,-1916,...,10,-676.00,-6760,-1547,-8,7,9414.642857,65902.5,0.0,13180.5


In [40]:
bureau_agg.drop(bureau_agg.iloc[:, 1:6], inplace = True, axis = 1) 
bureau_agg.head()

,SK_ID_CURR,bureau_DAYS_CREDIT_count,bureau_DAYS_CREDIT_mean,bureau_DAYS_CREDIT_sum,bureau_DAYS_CREDIT_min,bureau_DAYS_CREDIT_max,bureau_CREDIT_DAY_OVERDUE_count,bureau_CREDIT_DAY_OVERDUE_mean,bureau_CREDIT_DAY_OVERDUE_sum,bureau_CREDIT_DAY_OVERDUE_min,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_sum,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_max
0,100004,2,-867.000000,-1734,-1326,-408,2,0.0,0,0,...,2,-532.00,-1064,-682,-382,0,NaN,0.0,NaN,NaN
1,100054,2,-1310.500000,-2621,-1483,-1138,2,0.0,0,0,...,2,-578.00,-1156,-1114,-42,0,NaN,0.0,NaN,NaN
2,100055,7,-876.857143,-6138,-2202,-161,7,0.0,0,0,...,7,-288.00,-2016,-789,-13,0,NaN,0.0,NaN,NaN
3,100077,4,-824.250000,-3297,-1186,-148,4,0.0,0,0,...,4,-537.75,-2151,-773,-24,0,NaN,0.0,NaN,NaN
4,100181,10,-1041.300000,-10413,-1916,-137,10,0.0,0,0,...,10,-676.00,-6760,-1547,-8,7,9414.642857,65902.5,0.0,13180.5


In [41]:
model_df = model_df.merge(bureau_agg, on = ['SK_ID_CURR'], how = 'left')
model_df.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,...,bureau_DAYS_CREDIT_UPDATE_count,bureau_DAYS_CREDIT_UPDATE_mean,bureau_DAYS_CREDIT_UPDATE_sum,bureau_DAYS_CREDIT_UPDATE_min,bureau_DAYS_CREDIT_UPDATE_max,bureau_AMT_ANNUITY_count,bureau_AMT_ANNUITY_mean,bureau_AMT_ANNUITY_sum,bureau_AMT_ANNUITY_min,bureau_AMT_ANNUITY_max
0,272103,1,0.209155,0.383783,0.498982,9.325899,2,2,7.750184,9.323758,...,9.0,-318.888889,-2870.0,-1003.0,-8.0,0.0,NaN,0.0,NaN,NaN
1,245137,1,0.420611,0.176568,0.184767,9.226902,2,2,5.659482,8.420242,...,3.0,-235.666667,-707.0,-573.0,-67.0,0.0,NaN,0.0,NaN,NaN
2,169665,1,0.192942,0.298317,0.396168,9.846970,2,2,5.993961,9.166911,...,1.0,-8.000000,-8.0,-8.0,-8.0,0.0,NaN,0.0,NaN,NaN
3,159887,1,0.646330,0.298317,0.437300,9.545526,2,2,2.079442,9.132811,...,15.0,-737.666667,-11065.0,-2672.0,-16.0,0.0,NaN,0.0,NaN,NaN
4,206817,1,0.466864,0.298317,0.151982,9.815093,2,2,7.296413,7.817223,...,15.0,-847.133333,-12707.0,-2236.0,-3.0,0.0,NaN,0.0,NaN,NaN


In [42]:
categorical = pd.get_dummies(bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = bureau['SK_ID_CURR']
categorical.head()

,CREDIT_ACTIVE_Active,CREDIT_ACTIVE_Closed,CREDIT_ACTIVE_Sold,CREDIT_CURRENCY_currency 1,CREDIT_CURRENCY_currency 2,CREDIT_CURRENCY_currency 3,CREDIT_TYPE_Another type of loan,CREDIT_TYPE_Car loan,CREDIT_TYPE_Consumer credit,CREDIT_TYPE_Credit card,CREDIT_TYPE_Loan for business development,CREDIT_TYPE_Loan for working capital replenishment,CREDIT_TYPE_Microloan,CREDIT_TYPE_Mortgage,CREDIT_TYPE_Unknown type of loan,SK_ID_CURR
0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,400486
1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,400486
2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,400486
3,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,400486
4,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,400486


In [43]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
categorical_grouped.head()

CREDIT_ACTIVE_Active           CREDIT_ACTIVE_Closed            \
                            sum      mean                  sum      mean   
SK_ID_CURR                                                                 
100004                        0  0.000000                    2  1.000000   
100054                        1  0.500000                    1  0.500000   
100055                        4  0.571429                    3  0.428571   
100077                        1  0.250000                    3  0.750000   
100181                        2  0.200000                    8  0.800000   

           CREDIT_ACTIVE_Sold      CREDIT_CURRENCY_currency 1       \
                          sum mean                        sum mean   
SK_ID_CURR                                                           
100004                      0  0.0                          2  1.0   
100054                      0  0.0                          2  1.0   
100055                      0  0.0                          7  1.0   
100077                      0  0.0                          4  1.0   
100181                      0  0.0                         10  1.0   

           CREDIT_CURRENCY_currency 2       ...  \
                                  sum mean  ...   
SK_ID_CURR                                  ...   
100004                              0  0.0  ...   
100054                              0  0.0  ...   
100055                              0  0.0  ...   
100077                              0  0.0  ...   
100181                              0  0.0  ...   

           CREDIT_TYPE_Loan for business development       \
                                                 sum mean   
SK_ID_CURR                                                  
100004                                             0  0.0   
100054                                             0  0.0   
100055                                             0  0.0   
100077                                             0  0.0   
100181                                             0  0.0   

           CREDIT_TYPE_Loan for working capital replenishment       \
                                                          sum mean   
SK_ID_CURR                                                           
100004                                                      0  0.0   
100054                                                      0  0.0   
100055                                                      0  0.0   
100077                                                      0  0.0   
100181                                                      0  0.0   

           CREDIT_TYPE_Microloan      CREDIT_TYPE_Mortgage       \
                             sum mean                  sum mean   
SK_ID_CURR                                                        
100004                         0  0.0                    0  0.0   
100054                         0  0.0                    0  0.0   
100055                         0  0.0                    0  0.0   
100077                         0  0.0                    0  0.0   
100181                         0  0.0                    0  0.0   

           CREDIT_TYPE_Unknown type of loan       
                                        sum mean  
SK_ID_CURR                                        
100004                                    0  0.0  
100054                                    0  0.0  
100055                                    0  0.0  
100077                                    0  0.0  
100181                                    0  0.0  

[5 rows x 30 columns]

In [44]:
group_var = 'SK_ID_CURR'

# Need to create new column names
columns = []

# Iterate through the variables names
for var in categorical_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != group_var:
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical_grouped.columns = columns

categorical_grouped.head()

,CREDIT_ACTIVE_Active_count,CREDIT_ACTIVE_Active_count_norm,CREDIT_ACTIVE_Closed_count,CREDIT_ACTIVE_Closed_count_norm,CREDIT_ACTIVE_Sold_count,CREDIT_ACTIVE_Sold_count_norm,CREDIT_CURRENCY_currency 1_count,CREDIT_CURRENCY_currency 1_count_norm,CREDIT_CURRENCY_currency 2_count,CREDIT_CURRENCY_currency 2_count_norm,...,CREDIT_TYPE_Loan for business development_count,CREDIT_TYPE_Loan for business development_count_norm,CREDIT_TYPE_Loan for working capital replenishment_count,CREDIT_TYPE_Loan for working capital replenishment_count_norm,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Microloan_count_norm,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Mortgage_count_norm,CREDIT_TYPE_Unknown type of loan_count,CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100004,0,0.000000,2,1.000000,0,0.0,2,1.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100054,1,0.500000,1,0.500000,0,0.0,2,1.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100055,4,0.571429,3,0.428571,0,0.0,7,1.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100077,1,0.250000,3,0.750000,0,0.0,4,1.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100181,2,0.200000,8,0.800000,0,0.0,10,1.0,0,0.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [45]:
model_df = model_df.merge(categorical_grouped, left_on = 'SK_ID_CURR', right_index = True, how = 'left')
model_df.head()

,SK_ID_CURR,TARGET,EXT_SOURCE_3,EXT_SOURCE_1,EXT_SOURCE_2,DAYS_BIRTH,REGION_RATING_CLIENT_W_CITY,REGION_RATING_CLIENT,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,...,CREDIT_TYPE_Loan for business development_count,CREDIT_TYPE_Loan for business development_count_norm,CREDIT_TYPE_Loan for working capital replenishment_count,CREDIT_TYPE_Loan for working capital replenishment_count_norm,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Microloan_count_norm,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Mortgage_count_norm,CREDIT_TYPE_Unknown type of loan_count,CREDIT_TYPE_Unknown type of loan_count_norm
0,272103,1,0.209155,0.383783,0.498982,9.325899,2,2,7.750184,9.323758,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,245137,1,0.420611,0.176568,0.184767,9.226902,2,2,5.659482,8.420242,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,169665,1,0.192942,0.298317,0.396168,9.846970,2,2,5.993961,9.166911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,159887,1,0.646330,0.298317,0.437300,9.545526,2,2,2.079442,9.132811,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,206817,1,0.466864,0.298317,0.151982,9.815093,2,2,7.296413,7.817223,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
from my_functions import missing_value
missing_value(model_df)

,Missing Values,% of Total Values
bureau_AMT_ANNUITY_mean,7346,73.5
bureau_AMT_ANNUITY_min,7346,73.5
bureau_AMT_ANNUITY_max,7346,73.5
bureau_AMT_CREDIT_MAX_OVERDUE_mean,4129,41.3
bureau_AMT_CREDIT_MAX_OVERDUE_max,4129,41.3
bureau_AMT_CREDIT_MAX_OVERDUE_min,4129,41.3
bureau_DAYS_ENDDATE_FACT_max,2846,28.5
bureau_DAYS_ENDDATE_FACT_min,2846,28.5
bureau_DAYS_ENDDATE_FACT_mean,2846,28.5
bureau_AMT_CREDIT_SUM_LIMIT_max,2327,23.3


In [52]:
model_df=model_df.drop(columns=['bureau_AMT_ANNUITY_mean','bureau_AMT_ANNUITY_min','bureau_AMT_ANNUITY_max','bureau_AMT_CREDIT_MAX_OVERDUE_mean','bureau_AMT_CREDIT_MAX_OVERDUE_max','bureau_AMT_CREDIT_MAX_OVERDUE_min'])

In [53]:
model_df_corr=model_df.corr()
abs(model_df_corr.TARGET).sort_values(ascending=False)[:50]

TARGET                                1.000000
EXT_SOURCE_1                          0.509788
EXT_SOURCE_3                          0.366627
EXT_SOURCE_2                          0.261426
bureau_DAYS_CREDIT_mean               0.164069
CREDIT_ACTIVE_Closed_count_norm       0.159066
CREDIT_ACTIVE_Active_count_norm       0.156154
DAYS_BIRTH                            0.141339
DAYS_BIRTH_GROUP                      0.138141
bureau_DAYS_CREDIT_min                0.135967
bureau_DAYS_CREDIT_UPDATE_mean        0.132476
DAYS_EMPLOYED                         0.132066
CREDIT_ACTIVE_Active_count            0.123215
OWN_CAR_AGE                           0.119353
REGION_RATING_CLIENT_W_CITY           0.117141
REGION_RATING_CLIENT                  0.112320
BASEMENTAREA_MEDI                     0.099660
LIVINGAREA_AVG                        0.098726
bureau_DAYS_ENDDATE_FACT_min          0.098424
bureau_DAYS_ENDDATE_FACT_sum          0.096879
BASEMENTAREA_AVG                      0.096775
bureau_DAYS_C

In [56]:
selected_columns = ['TARGET','EXT_SOURCE_1','EXT_SOURCE_3','EXT_SOURCE_2','bureau_DAYS_CREDIT_mean', 'CREDIT_ACTIVE_Closed_count_norm','CREDIT_ACTIVE_Active_count_norm','DAYS_BIRTH','DAYS_BIRTH_GROUP','bureau_DAYS_CREDIT_min','bureau_DAYS_CREDIT_UPDATE_mean','DAYS_EMPLOYED','CREDIT_ACTIVE_Active_count','OWN_CAR_AGE','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','BASEMENTAREA_MEDI','LIVINGAREA_AVG','bureau_DAYS_ENDDATE_FACT_min','bureau_DAYS_ENDDATE_FACT_sum','BASEMENTAREA_AVG','bureau_DAYS_CREDIT_ENDDATE_sum','LIVINGAREA_MEDI','BASEMENTAREA_MODE','bureau_DAYS_CREDIT_max','TOTALAREA_MODE','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','FLAG_EMP_PHONE','LIVINGAREA_MODE','bureau_DAYS_CREDIT_sum','APARTMENTS_AVG','bureau_DAYS_CREDIT_UPDATE_sum','APARTMENTS_MODE','bureau_DAYS_ENDDATE_FACT_mean','APARTMENTS_MEDI','CREDIT_TYPE_Microloan_count_norm','bureau_DAYS_CREDIT_ENDDATE_mean','REG_CITY_NOT_WORK_CITY','bureau_DAYS_CREDIT_UPDATE_min','REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','bureau_DAYS_CREDIT_ENDDATE_min','FLOORSMAX_AVG','CREDIT_TYPE_Credit card_count_norm','AMT_CREDIT','FLOORSMAX_MEDI','CREDIT_TYPE_Credit card_count','ELEVATORS_AVG','FLOORSMAX_MODE']
model_df = select_columns(model_df, selected_columns)

In [59]:
missing_value(model_df)

,Missing Values,% of Total Values


In [58]:
model_df['bureau_DAYS_ENDDATE_FACT_min'].fillna((model_df['bureau_DAYS_ENDDATE_FACT_min'].median()), inplace=True)
model_df['bureau_DAYS_ENDDATE_FACT_mean'].fillna((model_df['bureau_DAYS_ENDDATE_FACT_mean'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_ENDDATE_min'].fillna((model_df['bureau_DAYS_CREDIT_ENDDATE_min'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_ENDDATE_mean'].fillna((model_df['bureau_DAYS_CREDIT_ENDDATE_mean'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_mean'].fillna((model_df['bureau_DAYS_CREDIT_mean'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_sum'].fillna((model_df['bureau_DAYS_CREDIT_sum'].median()), inplace=True)
model_df['CREDIT_TYPE_Credit card_count_norm'].fillna((model_df['CREDIT_TYPE_Credit card_count_norm'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_UPDATE_min'].fillna((model_df['bureau_DAYS_CREDIT_UPDATE_min'].median()), inplace=True)
model_df['CREDIT_TYPE_Microloan_count_norm'].fillna((model_df['CREDIT_TYPE_Microloan_count_norm'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_UPDATE_sum'].fillna((model_df['bureau_DAYS_CREDIT_UPDATE_sum'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_max'].fillna((model_df['bureau_DAYS_CREDIT_max'].median()), inplace=True)
model_df['CREDIT_ACTIVE_Closed_count_norm'].fillna((model_df['CREDIT_ACTIVE_Closed_count_norm'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_ENDDATE_sum'].fillna((model_df['bureau_DAYS_CREDIT_ENDDATE_sum'].median()), inplace=True)
model_df['bureau_DAYS_ENDDATE_FACT_sum'].fillna((model_df['bureau_DAYS_ENDDATE_FACT_sum'].median()), inplace=True)
model_df['CREDIT_ACTIVE_Active_count'].fillna((model_df['CREDIT_ACTIVE_Active_count'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_UPDATE_mean'].fillna((model_df['bureau_DAYS_CREDIT_UPDATE_mean'].median()), inplace=True)
model_df['bureau_DAYS_CREDIT_min'].fillna((model_df['bureau_DAYS_CREDIT_min'].median()), inplace=True)
model_df['CREDIT_ACTIVE_Active_count_norm'].fillna((model_df['CREDIT_ACTIVE_Active_count_norm'].median()), inplace=True)
model_df['CREDIT_TYPE_Credit card_count'].fillna((model_df['CREDIT_TYPE_Credit card_count'].median()), inplace=True)

In [60]:
Y=model_df['TARGET']
X=model_df[['EXT_SOURCE_1','EXT_SOURCE_3','EXT_SOURCE_2','bureau_DAYS_CREDIT_mean', 'CREDIT_ACTIVE_Closed_count_norm','CREDIT_ACTIVE_Active_count_norm','DAYS_BIRTH','DAYS_BIRTH_GROUP','bureau_DAYS_CREDIT_min','bureau_DAYS_CREDIT_UPDATE_mean','DAYS_EMPLOYED','CREDIT_ACTIVE_Active_count','OWN_CAR_AGE','REGION_RATING_CLIENT_W_CITY','REGION_RATING_CLIENT','BASEMENTAREA_MEDI','LIVINGAREA_AVG','bureau_DAYS_ENDDATE_FACT_min','bureau_DAYS_ENDDATE_FACT_sum','BASEMENTAREA_AVG','bureau_DAYS_CREDIT_ENDDATE_sum','LIVINGAREA_MEDI','BASEMENTAREA_MODE','bureau_DAYS_CREDIT_max','TOTALAREA_MODE','REG_CITY_NOT_LIVE_CITY','FLAG_DOCUMENT_3','FLAG_EMP_PHONE','LIVINGAREA_MODE','bureau_DAYS_CREDIT_sum','APARTMENTS_AVG','bureau_DAYS_CREDIT_UPDATE_sum','APARTMENTS_MODE','bureau_DAYS_ENDDATE_FACT_mean','APARTMENTS_MEDI','CREDIT_TYPE_Microloan_count_norm','bureau_DAYS_CREDIT_ENDDATE_mean','REG_CITY_NOT_WORK_CITY','bureau_DAYS_CREDIT_UPDATE_min','REGION_POPULATION_RELATIVE','DAYS_REGISTRATION','bureau_DAYS_CREDIT_ENDDATE_min','FLOORSMAX_AVG','CREDIT_TYPE_Credit card_count_norm','AMT_CREDIT','FLOORSMAX_MEDI','CREDIT_TYPE_Credit card_count','ELEVATORS_AVG','FLOORSMAX_MODE']]

In [61]:
lrm=linear_model.LogisticRegression()
lrm.fit(X,Y)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [62]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 TARGET   R-squared:                       0.375
Model:                            OLS   Adj. R-squared:                  0.372
Method:                 Least Squares   F-statistic:                     121.9
Date:                Tue, 18 Aug 2020   Prob (F-statistic):               0.00
Time:                        16:57:25   Log-Likelihood:                -4907.0
No. Observations:               10000   AIC:                             9914.
Df Residuals:                    9950   BIC:                         1.027e+04
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
const                                 -1.3059      0.506     -2.581      0.010      -2.298      -0.314
EXT_SOURCE_1                          -2.0202      0.041    -48.906      0.000      -2.101      -1.939
EXT_SOURCE_3                          -0.6365      0.025    -25.042      0.000      -0.686      -0.587
EXT_SOURCE_2                          -0.4569      0.029    -15.904      0.000      -0.513      -0.401
bureau_DAYS_CREDIT_mean            -3.543e-05   3.61e-05     -0.980      0.327      -0.000    3.54e-05
CREDIT_ACTIVE_Closed_count_norm       -0.0789      0.116     -0.682      0.495      -0.306       0.148
CREDIT_ACTIVE_Active_count_norm       -0.0553      0.118     -0.470      0.638      -0.286       0.175
DAYS_BIRTH                             0.3201      0.048      6.633      0.000       0.226       0.415
DAYS_BIRTH_GROUP                       0.0251      0.013      1.858      0.063      -0.001       0.052
bureau_DAYS_CREDIT_min             -8.056e-06   1.64e-05     -0.492      0.623   -4.02e-05     2.4e-05
bureau_DAYS_CREDIT_UPDATE_mean      4.088e-05   2.89e-05      1.416      0.157   -1.57e-05    9.75e-05
DAYS_EMPLOYED                         -0.0168      0.004     -3.800      0.000      -0.026      -0.008
CREDIT_ACTIVE_Active_count             0.0089      0.005      1.655      0.098      -0.002       0.020
OWN_CAR_AGE                            0.0043      0.001      7.776      0.000       0.003       0.005
REGION_RATING_CLIENT_W_CITY            0.0739      0.026      2.812      0.005       0.022       0.125
REGION_RATING_CLIENT                  -0.0434      0.026     -1.680      0.093      -0.094       0.007
BASEMENTAREA_MEDI                    -15.1905      1.776     -8.555      0.000     -18.671     -11.710
LIVINGAREA_AVG                        -4.6249      1.079     -4.287      0.000      -6.740      -2.510
bureau_DAYS_ENDDATE_FACT_min        1.803e-05   1.61e-05      1.117      0.264   -1.36e-05    4.97e-05
bureau_DAYS_ENDDATE_FACT_sum       -7.839e-06    6.4e-06     -1.225      0.221   -2.04e-05     4.7e-06
BASEMENTAREA_AVG                      15.5815      1.805      8.634      0.000      12.044      19.119
bureau_DAYS_CREDIT_ENDDATE_sum      5.091e-07   6.61e-07      0.770      0.441   -7.87e-07    1.81e-06
LIVINGAREA_MEDI                        1.6656      1.078      1.544      0.123      -0.448       3.780
BASEMENTAREA_MODE                     -0.9501      0.562     -1.690      0.091      -2.052       0.152
bureau_DAYS_CREDIT_max             -2.274e-05   1.86e-05     -1.222      0.222   -5.92e-05    1.37e-05
TOTALAREA_MODE                         0.2548      0.190      1.344      0.179      -0.117       0.626
REG_CITY_NOT_LIVE_CITY                 0.0484      0.015      3.244      0.001       0.019      

In [65]:
X_train, X_test, y_train, y_test =  train_test_split(X, Y, test_size=0.20, random_state=111)
second_model = LogisticRegression()
second_model.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [66]:
prediction_train = second_model.predict(X_train)
prediction_test = second_model.predict(X_test)

In [67]:
print("recall_score() değeri        : ",  recall_score(y_test,prediction_test))

recall_score() değeri        :  0.502127659574468


## Winsorize

In [68]:
import seaborn as sns
def outlier_inspect(df, col, min_z=1, max_z = 5, step = 0.5, max_hist = None, bins = 50):
    fig = plt.figure(figsize=(20, 6))
    fig.suptitle(col, fontsize=16)
    plt.subplot(1,3,1)
    if max_hist == None:
        sns.distplot(df[col], kde=False, bins = 50)
    else :
        sns.distplot(df[df[col]<=max_hist][col], kde=False, bins = 50)
    plt.subplot(1,3,2)
    sns.boxplot(df[col])
    plt.subplot(1,3,3)
    z_score_inspect = outlier_zscore(df, col, min_z=min_z, max_z = max_z, step = step)
    plt.show()

In [ ]:
outlier_inspect(model_logged,'EXT_SOURCE_1')
outlier_inspect(model_logged,'DAYS_LAST_PHONE_CHANGE')
outlier_inspect(model_logged,'DAYS_REGISTRATION')
outlier_inspect(model_logged,'AMT_GOODS_PRICE')
outlier_inspect(model_logged,'DAYS_ID_PUBLISH')
outlier_inspect(model_logged,'FLOORSMAX_AVG')
outlier_inspect(model_logged,'FLOORSMAX_MEDI')
outlier_inspect(model_logged,'FLOORSMAX_MODE')
outlier_inspect(model_logged,'FLOORSMIN_AVG')
outlier_inspect(model_logged,'FLOORSMIN_MEDI')
outlier_inspect(model_logged,'FLOORSMIN_MODE')

## Pandas Profiling Report

In [ ]:
from pandas_profiling import ProfileReport
profile = ProfileReport(train_model, title="Pandas Profiling Report")

In [ ]:
profile.to_file("home_credit_profiling.html")